## 1 Autorizar el acceso necesario a Earth Engine e importar librerias

In [10]:
import ee
ee.Authenticate()
ee.Initialize()

In [11]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import sys
from pathlib import Path
import matplotlib.pyplot as plt

## 2. Funciones

In [12]:
# 2.1
def ee_array_to_data(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    data = pd.DataFrame(arr)

    # Rearrange the header.
    headers = data.iloc[0]
    data    = pd.DataFrame(data.values[1:], columns=headers)

    # Remove rows without data inside.
    data = data[['id','longitude', 'latitude', 'time', *list_of_bands]]

    # Convert the data to numeric values.
    for band in list_of_bands:
        data[band] = pd.to_numeric(data[band], errors='coerce')

    # Convert the time field into a datetime.
    data['datetime'] = pd.to_datetime(data['time'], unit='ms')

    # Keep the columns of interest.
    data = data[['time','datetime',  *list_of_bands]]
    
    return data

In [13]:
# 2.2 Date format
def dates_dt(date_years):
    start_date = dt.date(date_years[0],    1, 1)
    end_date   = dt.date(date_years[1]+1, 1, 1)
    dates      = [start_date + dt.timedelta(n) for n in range(int ((end_date - start_date).days))]     
    return dates

In [14]:
# 2.3 Access GEE data:"NASA/GDDP-CMIP6" and  extract series (nearest grid)
def extract_GEE_NEX_GDDP(var, date, GCM_EE_model_names, scenario, estaciones_lista, name_stations):
    #VER GEE
    GCM_EE     = ee.ImageCollection("NASA/GDDP-CMIP6")    
    estaciones = ee.FeatureCollection(estaciones_lista)
    
    # Set date target
    start_date = f'{date[0]}-01-01'
    end_date = f'{date[1]+1}-01-01'
    
    GEE_EE_data = []
    
    # Filter and storage data in dataframe
    for i in range(len(GCM_EE_model_names)):    
        GCM_EE_filter = GCM_EE.select(var).filter(
                        ee.Filter.And(ee.Filter.eq('scenario', scenario),
                        ee.Filter.eq('model', GCM_EE_model_names[i]),
                        ee.Filter.date(start_date, end_date)))
        
        extraction = GCM_EE_filter.getRegion(estaciones, 100).getInfo() 
        arr_data = ee_array_to_data(extraction,[var])
        if var == 'pr':
            data_arr = pd.DataFrame(np.reshape(arr_data[var].values,(len(estaciones_lista),-1))).transpose()*86400
        data_arr.columns=name_stations
        data_arr['date'] = arr_data.datetime.unique()
        data_arr = data_arr.set_index('date')
        GEE_EE_data.append(data_arr)
    
    return GEE_EE_data

In [15]:
# 2.4 Quantile Delta Mapping function
# Por: Dr. Martin Gomez-Garcia
def QDM(var, dailyO, dailyMC, dailyMT, datesO, datesC, datesT):
         
    startYear, endYear = datesO[0].year, datesO[-1].year
    CDFO               = np.zeros((12,31*3*(endYear-startYear+1)),dtype = np.float64) #*3 window 3 months   
    startYear, endYear = datesC[0].year, datesC[-1].year
    CDFMC              = np.zeros((12,31*3*(endYear-startYear+1)),dtype = np.float64)
    startYear, endYear = datesT[0].year, datesT[-1].year
    CDFMT              = np.zeros((12,31*3*(endYear-startYear+1)),dtype = np.float64)

    arrLenCO,arrLenC,arrLenT = np.zeros((3,12),dtype = np.int32)
 
    # For precipitation  
    if var == 'pr':

        for im in range(1,13):
                
            # Define the months for calibration
            monBef = 12 if im == 1  else im-1
            monAct = im
            monAft = 1  if im == 12 else im+1    

            indxsCO = np.where([(dt.month==monBef or dt.month==monAct or dt.month==monAft)for dt in datesO])
            indxsC  = np.where([(dt.month==monBef or dt.month==monAct or dt.month==monAft)for dt in datesC])
            arrLenCO[im-1] = len(indxsCO[0])
            arrLenC[im-1]  = len(indxsC[0])

            indxsT  = np.where([(dt.month==monBef or dt.month==monAct or dt.month==monAft)for dt in datesT])
            arrLenT[im-1] = len(indxsT[0])

            # Obs
            tempDaily = np.sort(dailyO[indxsCO])
            numZeros = len(np.where(tempDaily<0.05)[0])
            randNum  = np.random.uniform(0.00001,0.05,size=numZeros)
            tempDaily[np.where(tempDaily<0.05)] = np.sort(randNum)
            CDFO[im-1][:arrLenCO[im-1]] = tempDaily    

            # Mod Calib
            tempDaily = np.sort(dailyMC[indxsC])
            numZeros = len(np.where(tempDaily<0.05)[0])
            randNum = np.random.uniform(0.00001,0.05,size=numZeros)
            tempDaily[np.where(tempDaily<0.05)] = np.sort(randNum)
            CDFMC[im-1][:arrLenC[im-1]] = tempDaily

            # Mod Target
            tempDaily = np.sort(dailyMT[indxsT])
            numZeros = len(np.where(tempDaily<0.05)[0])
            randNum  = np.random.uniform(0.00001,0.05,size = numZeros)
            tempDaily[np.where(tempDaily<0.05)] = np.sort(randNum)
            CDFMT[im-1][:arrLenT[im-1]] = tempDaily

        tildeDaily = np.zeros_like(dailyO)
        for ii,iDay in enumerate(datesO):
                
            # Loop over each day in the target period : datesT . Determine this month.
            im = iDay.month
            # Copy the CDFs
            ecdfO  =  CDFO[im-1][:arrLenCO[im-1]]
            ecdfMC = CDFMC[im-1][:arrLenC[im-1]]
            ecdfMT = CDFMT[im-1][:arrLenT[im-1]]

            # Create a probability array for the calibration period (pCO;pC) and target period (pT)
            pCO = (np.arange(arrLenCO[im-1])-0.5)/arrLenCO[im-1]
            pC  = (np.arange(arrLenC[im-1])-0.5)/arrLenC[im-1]
            pT  = (np.arange(arrLenT[im-1])-0.5)/arrLenT[im-1]

            # Calculate the absolute change of the quantile (qDelta)
            thisQuantile = np.interp(dailyO[ii],ecdfO,pCO)
            daily_MC_q     = np.interp(thisQuantile,pC,ecdfMC)
            daily_MT_q     = np.interp(thisQuantile,pT,ecdfMT)

            qDelta       = daily_MT_q / daily_MC_q    
            # Calculate the corrected value
            tildeDaily[ii] = np.interp(thisQuantile,pCO,ecdfO) * qDelta 
            tildeDaily[np.where(tildeDaily<0.05)] = 0   
       
        for im in range(1,13):
                
            indxsO = np.where([dt.month==im for dt in datesO])
            indxsC = np.where([dt.month==im for dt in datesC])
            indxsT = np.where([dt.month==im for dt in datesT])

            thres     = 1.9*np.max(dailyMT[indxsT])*np.max(dailyO[indxsO])/np.max(dailyMC[indxsC])

            #print(np.max(dailyMT[indxsT]),np.max(dailyO[indxsO]),np.max(dailyMC[indxsC]))
            tempTildes = tildeDaily[indxsT]
            tempTildes[np.where(tempTildes>thres)] = thres
            tildeDaily[indxsT] = tempTildes                    
   
    
    return tildeDaily

In [16]:
# 2.5 Create output folder
def output_folder_directory(var, outputh_folder_path):
    # Create output folte if it does not exist
    output_path = os.path.join(outputh_folder_path, var)
    isExist = os.path.exists(output_path)

    if isExist == False:
        os.makedirs(output_path) 
    
    os.chdir(output_path)
    return output_path

In [17]:
# 2.5 Automatizar proceso
def QDM_NEXGDDP(path_coords, path_data, var, step_time, ref_date, calibr_date, target_date,
                scenario, GCM_EE_model_names, output_path):
    path_file = Path(sys.path[0])
    os.chdir(path_file)
    print('Processing...')
    # Read data
    estaciones_data = pd.read_csv(path_coords, encoding='latin8')
    observed_data   = pd.read_csv(path_data, encoding='latin8')    

    # Coordinates and name stations
    estaciones_lista=[]
    for row in range(len(estaciones_data.index)):
        points=ee.Feature(ee.Geometry.Point(estaciones_data.longitud[row],
                                            estaciones_data.latitude[row]),
                         {'name':(estaciones_data.station_name[row])})
        estaciones_lista.append(points)

    name_stations = estaciones_data.station_name.to_list()        

    # Set dates   
    datesO = dates_dt(ref_date)
    datesC = dates_dt(calibr_date)
    datesT = dates_dt(target_date)
    
    # Historical
    scenario_hist = 'historical'
    GEE_EE_hist_data = extract_GEE_NEX_GDDP(var, calibr_date, GCM_EE_model_names, scenario_hist, estaciones_lista, name_stations)
    
    # Future
    scenario_target = scenario
    GEE_EE_target_data = extract_GEE_NEX_GDDP(var, target_date, GCM_EE_model_names, scenario, estaciones_lista, name_stations)
        
    output_folder = output_folder_directory(var, output_path)
    os.chdir(output_folder)
        
    future_qdm_dates = pd.date_range(f'{ref_date[0]}-01-01', f'{ref_date[1]}-12-31', freq=step_time)
    
    for i in range(len(GCM_EE_model_names)):
        
        qdm_series_sd_df = pd.DataFrame({'date': pd.to_datetime(future_qdm_dates)})
        historical_data = GEE_EE_hist_data[i]
        future_data     = GEE_EE_target_data[i]

        for station in name_stations:
            
            dailyO  = np.array(observed_data[f'{station}'])
            dailyMC = np.array(historical_data[f'{station}'])
            dailyMT = np.array(future_data[f'{station}'])
            qdm_sd  = QDM(var, dailyO, dailyMC, dailyMT, datesO, datesC, datesT)
            qdm_series_sd_df[f'{station}'] = qdm_sd  
            os.chdir(output_folder)            
        
        qdm_series_sd_df.to_csv(f'qdm_{var}_{GCM_EE_model_names[i]}_{scenario}_{target_date[0]}_{target_date[1]}.csv'
                                ,encoding='latin8'
                                ,index=False)      
        
        
        print('Done : ', f'{var}_{GCM_EE_model_names[i]}_{scenario}_{target_date[0]}_{target_date[1]}')

## 3. Downscaling precipitación

## 3.1 (1) SCENARIO

In [22]:
path_coords = r"C:\Users\Carolina\CC_EMT\estaciones_ichurata.csv"
path_data   = r"C:\Users\Carolina\CC_EMT\pcp_ichurata.csv"
var = 'pr'
step_time   = 'd'

ref_date    = [1976,2022]
calibr_date = [1970,2014]
target_date = [2036,2065]

scenario = 'ssp585'

GCM_EE_model_names=['GFDL-ESM4']

output_path = r"D:\00_OUTPUT_CC_EMT"

QDM_NEXGDDP(path_coords, path_data, var, step_time, ref_date, calibr_date, target_date,
            scenario, GCM_EE_model_names, output_path)

Processing...
Done :  pr_GFDL-ESM4_ssp585_2036_2065


C:\Users\Carolina\AppData\Local\Temp\ipykernel_2256\3619128213.py:84: RuntimeWarning: invalid value encountered in scalar divide
  thres     = 1.9*np.max(dailyMT[indxsT])*np.max(dailyO[indxsO])/np.max(dailyMC[indxsC])


In [20]:
path_coords = r"C:\Users\Carolina\CC_EMT\estaciones_rocha.csv"
path_data   = r"C:\Users\Carolina\CC_EMT\pcp_rocha.csv"
var = 'pr'
step_time   = 'd'

ref_date    = [1970,2021]
calibr_date = [1950,2014]
target_date = [2036,2065]
scenario    = 'ssp585'
GCM_EE_model_names = ['MIROC6']
output_path = r"D:\00_OUTPUT_CC_EMT"


QDM_NEXGDDP(path_coords, path_data, var, step_time, ref_date, calibr_date, target_date,
            scenario, GCM_EE_model_names, output_path)

Processing...
Done :  pr_MIROC6_ssp585_2036_2065
